In this notebook, we analyze an object in free fall under constant acceleration. As a first demonstration of the Kalman filter, there are no hidden variables. Said another way, the measurement vector are the same variables as the state vector.

In [10]:
using LinearAlgebra
using Random
using PlotlyJS

include("kalmanflt.jl");

WebIO._IJuliaInit()

The system is a particle falling due to gravity without air resistance. The equation of state are:

$$
\begin{align}
h &= h_{0} + v_{0} t + \frac{1}{2} a t^{2} \\
v &= v_{0} + a t
\end{align}
$$

In [11]:
time_steps = 125
delta_time_sec = 0.02  # sec
accel_ms2 = 9.8;  # meter / sec / sec

In [12]:
"""
    get_model_estimate(initial_state, measurement_time_sec)

Assuming constant acceleration, compute how the position and velocity change from their initial state

# Arguments
- `initial_state::Vector`: Must be a 2-item vector exactly as `Vector{Float64}([p, v])` where `p` is the position and `v` is the velocity
- `measurement_time_sec::Number`: The number of seconds from the initial state

# Returns
`Vector` of the state estimate

# Examples
```jldoctest
julia> v = get_model_estimate(Vector([9.8, 0.]), sqrt(2.))
2-element Vector{Float64}:
  -1.7763568394002505e-15
 -13.859292911256333
```
"""
function get_model_estimate(
    initial_state::Vector,
    measurement_time_sec::Number)::Vector
    
    transition_mat = [1.0 measurement_time_sec; 
                      0.0 1.0]
    external_input_vector = Vector([-accel_ms2, -accel_ms2])
    control_mat = [0.5 * measurement_time_sec^2 0.0; 
                   0.0 measurement_time_sec]
    return ((transition_mat * initial_state)
            + (control_mat * external_input_vector))
end;

Initialize the initial and true states

In [13]:
initial_state_vector = Vector([1000., 0.])
true_state_vector = Vector([1015., 2.5]);

Populate an array of the true state evolution in time

In [14]:
true_state_fcn_time = zeros(Float64, (2, time_steps))
for time_step_index = 1:time_steps
    time_sec = delta_time_sec * (time_step_index - 1)
    true_state_fcn_time[:, time_step_index] = get_model_estimate(true_state_vector, time_sec)
end;

Plot the true position

In [15]:
time_axis_sec = [delta_time_sec * (t-1) for t in 1:time_steps]

plot(
    scatter(
        mode="markers",
        x=time_axis_sec,
        y=true_state_fcn_time[1, :],
        name="True Position"
    ),
    Layout(
        title="True Position",
        xaxis_title="Time (second)",
        yaxis_title="Position (meters)",
        legend_title="Legend Title",
    ),
    config=PlotConfig(scrollZoom=true)
)

data: [
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields legend, margin, template, title, xaxis, and yaxis"

Plot the true velocity

In [16]:
plot(
    scatter(
        mode="markers",
        x=time_axis_sec,
        y=true_state_fcn_time[2, :],
        name="True Position"
    ),
    Layout(
        title="True Velocity",
        xaxis_title="Time (second)",
        yaxis_title="Velocity (meters / second)",
        legend_title="Legend Title",
    ),
    config=PlotConfig(scrollZoom=true),
)

data: [
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields legend, margin, template, title, xaxis, and yaxis"

In [17]:
"""
    get_measurement(true_state, measurement_time_sec)

Simulate the measurement at the specified time given the true state. Artificial noise is added to simulate a real system

# Arguments
- `true_state::Vector`: Must be a 2-item vector exactly as `Vector{Float64}([p, v])` where `p` is the position and `v` is the velocity
- `measurement_time_sec::Number`: The number of seconds from the true starting state

# Returns
`Vector` of the measurement

See also [`get_model_estimate`](@ref) for an example
"""
function get_measurement(
    true_state::Vector,
    measurement_time_sec)::Vector
    """Get the next measurement
    In a 'real' physical system, we would not need to simulate the measurement
    """

    model_estimate_vec = get_model_estimate(
        true_state, measurement_time_sec
    )
    # Pseudo-measurement using classical physics calculation with noise
    position_noise_m = 1.0  # meter
    velocity_noise_ms = 0.5  # meter / sec
    artificial_noise_vec = Vector([position_noise_m * randn(1); 
                                   velocity_noise_ms * randn(1)]
    )
    measurement::Vector = model_estimate_vec + artificial_noise_vec
    return measurement
end;

The state vector is position, velocity. We will use very generous covariance values here as values too small provide no little room for the algorithm to work.

In [18]:
initial_state_cov = Diagonal([100.0^2, # meters^2
                              10.0^2])  # (meters / seconds)^2
initial_state = kfstate{Float64}(initial_state_vector, initial_state_cov);

In [19]:
state_transition_mat = [1.0 delta_time_sec; 
                        0.0 1.0]
control_mat = [0.5 * delta_time_sec^2 0; 
               0                      delta_time_sec]
system_noise_mat = [0.1  0.05;
                    0.05 0.01]
external_input_vector = -1 * Vector([accel_ms2, accel_ms2])
system = kfsystem{Float64}(state_transition_mat, control_mat, system_noise_mat);

Initialize the observation matrix. The measurement and state is one-to-one. For the measurement, the variance for the position (velocity) is 100 meters-squared (1 meters-squared/sec-squared)

$$
\begin{align}
z_{n} &= H x_{n} \\
\begin{pmatrix}
h_{z_{n}} \\
v_{z_{n}}
\end{pmatrix} &= \begin{pmatrix}
1 & 0\\
0 & 1
\end{pmatrix}
\begin{pmatrix}
h_{x_{n}} \\
v_{x_{n}}
\end{pmatrix} \\
\text{cov}\left(z\right) &= 
\begin{pmatrix}
10^2 & 0 \\
0   & 1^2
\end{pmatrix}
\end{align}
$$

In [20]:
obs_mat = I(2)
obsCov = Diagonal([10^2, 5^2])
observations = kfobservation{Float64}(obs_mat, obsCov);

Gather measurements and update the state

In [21]:
states_fcn_time = Vector{kfstate{Float64}}(undef, time_steps);
# This is the seed for the algorithm
next_state = kfstate{Float64}(initial_state.state, initial_state.cov)

for step in 1:time_steps
    # At current time, get measurement
    time_sec = (step - 1) * delta_time_sec
    measurement_vec = get_measurement(true_state_vector, time_sec)
    true_state_vec = get_model_estimate(true_state_vector, time_sec)

    # Predict the next state
    predicted_state = extrapulate_state(next_state, system, external_input_vector)

    # Update the estimate
    next_state = update_state(predicted_state, observations, measurement_vec)
    states_fcn_time[step] = next_state

end;

Plot the true and estimated position

In [22]:
plot(
    [
        # True position
        scatter(
            mode="markers",
            x=time_axis_sec,
            y=true_state_fcn_time[1, :],
            name="True Position"
        ),
        # Estimated position
        scatter(
            mode="markers",
            x=time_axis_sec,
            y=[s.state[1] for s in states_fcn_time],
            error_y=attr(
                type="data",
                array=[sqrt(s.cov[1, 1]) for s in states_fcn_time]
            ),
            name="Estimated Position"
        )
    ],
    Layout(
        title="Position Measurement",
        xaxis_title="Time (second)",
        yaxis_title="Position (meters)",
        legend_title="Legend Title",
    ),
    config=PlotConfig(scrollZoom=true)
)

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields error_y, mode, name, type, x, and y"
]

layout: "layout with fields legend, margin, template, title, xaxis, and yaxis"

Plot the true and estimated velocity

In [23]:
plot(
    [
        # True velocity
        scatter(
            mode="markers",
            x=time_axis_sec,
            y=true_state_fcn_time[2, :],
            name="True Velocity"
        ),
        # Estimated velocity
        scatter(
            mode="markers",
            x=time_axis_sec,
            y=[s.state[2] for s in states_fcn_time],
            error_y=attr(
                type="data",
                array=[sqrt(s.cov[2, 2]) for s in states_fcn_time]
            ),
            name="Estimated Velocity"
        )
    ],
    Layout(
        title="Velocity Measurement",
        xaxis_title="Time (second)",
        yaxis_title="Velocity (meters / second)",
        legend_title="Legend Title",
    ),
    config=PlotConfig(scrollZoom=true)
)

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields error_y, mode, name, type, x, and y"
]

layout: "layout with fields legend, margin, template, title, xaxis, and yaxis"